# Import necessary libraries.

In [1]:
# import libraries
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
# import geopandas as gpd

---
# Import & concatenate data

Read all CSV files from the data folder.

In [2]:
# get CSV files list from data folder with sorted order
csv_files = glob.glob("data/*.csv")

Concatenate all DataFrames into one DataFrame.

In [3]:
# read each CSV file into DataFrame
# this creates a list of DataFrames

df_list = []
for file in csv_files:
    df = pd.read_csv(file, keep_default_na=False)
    
    # add date column
    df['Date'] = file[5:-4]
    df.Date = pd.to_datetime(df.Date)
    
    # drop continent rows
    df.drop(df.index[0:7], inplace=True)
    df_list.append(df)

# concatenate all DataFrames
big_df = pd.concat(df_list, ignore_index=True)

#DataFrame not containing continent rows
continents = ['Asia','North America','Europe', 'South America', 'Africa', 'Oceania']

In [4]:
big_df.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,Date
0,World,"679,494,398","117,336","6,797,254",679,"652,277,902","120,702","20,419,242","40,559","87,173",872,,,,All,2023-02-24
1,China,"503,302",,"5,272",,"379,053",,"118,977","7,557",347,4,"160,000,000","110,461","1,448,471,400",Asia,2023-02-24
2,USA,"105,169,945","8,922","1,144,441",73,"102,567,861","34,979","1,457,643","2,588","314,123","3,418","1,164,027,866","3,476,731","334,805,269",North America,2023-02-24
3,India,"44,685,799",180,"530,764",,"44,152,945",106,"2,090",N/A,"31,768",377,"918,324,498","652,854","1,406,631,776",Asia,2023-02-24
4,France,"39,608,954","3,917","164,877",29,"39,374,920","3,293","69,157",869,"603,938","2,514","271,490,188","4,139,547","65,584,518",Europe,2023-02-24
5,Germany,"38,111,063","20,974","167,812",89,"37,654,800",,"288,451",N/A,"454,333","2,001","122,332,384","1,458,359","83,883,596",Europe,2023-02-24
6,Brazil,"37,020,531","11,587","698,928",94,"36,185,975","6,329","135,628",N/A,"171,906","3,245","63,776,166","296,146","215,353,593",South America,2023-02-24
7,Japan,"33,157,721","6,512","72,134",83,"21,675,630","2,474","11,409,957",183,"264,026",574,"94,827,129","755,084","125,584,838",Asia,2023-02-24
8,S. Korea,"30,479,753","10,051","33,929",20,"30,229,352","5,828","216,472",183,"593,801",661,"15,804,065","307,892","51,329,899",Asia,2023-02-24
9,Italy,"25,576,852",,"188,094",,"25,219,653",,"169,105",133,"424,422","3,121","267,663,187","4,441,601","60,262,770",Europe,2023-02-24


---
# Data exploration:

## Feature information:

| Header name      | Description                     |
|------------------|--------------------------------:|
| Country,Other    | Name of the country |
| TotalCases       | Total number of cases           |
| NewCases         | New cases of the day            |
| TotalDeaths      | Total number of deaths          |
| NewDeaths        | New deaths of the day           |
| TotalRecovered   | Total number of recovered cases |
| NewRecovered     | New recovered cases of the day  |
| ActiveCases      | Active cases (F0, F1, ...) |
| Serious,Critical | Serious cases                   |
| TotCases/1M pop  | Total cases per 1M population   |    
| Deaths/1M pop    | Total deaths per 1M population  |
| TotalTests       | Total number of tests           |
| Tests/1M pop     | Total tests per 1M population   |
| Population       | Population of the country       |
| Continent        | Continent of the country        |

## Number of rows and columns of the DataFrame.

In [5]:
# get number of rows
num_rows = big_df.shape[0]

# get number of columns
num_cols = big_df.shape[1]

print('Number of rows:', num_rows)
print('Number of columns:', num_cols)


Number of rows: 4872
Number of columns: 16


## Data type

In [6]:
# get DataFrame info
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4872 entries, 0 to 4871
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Country,Other     4872 non-null   object        
 1   TotalCases        4872 non-null   object        
 2   NewCases          4872 non-null   object        
 3   TotalDeaths       4872 non-null   object        
 4   NewDeaths         4872 non-null   object        
 5   TotalRecovered    4872 non-null   object        
 6   NewRecovered      4872 non-null   object        
 7   ActiveCases       4872 non-null   object        
 8   Serious,Critical  4872 non-null   object        
 9   TotCases/1M pop   4872 non-null   object        
 10  Deaths/1M pop     4872 non-null   object        
 11  TotalTests        4872 non-null   object        
 12  Tests/1M pop      4872 non-null   object        
 13  Population        4872 non-null   object        
 14  Continent         4872 n

Comment: the type of the columns is `object`, so we need to convert them to `numeric`.

## Check duplicates rows.

In [7]:
# get number of duplicated rows
duplicated_rows = big_df.duplicated().sum()

# check if there are duplicated rows
if duplicated_rows > 0:
    print("Number of duplicated rows:", duplicated_rows)
else:
    print("No duplicated rows found")


No duplicated rows found


## Some statistical:

In [8]:
big_df.describe().T

C:\Users\doana\AppData\Local\Temp\ipykernel_16152\2142895519.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  big_df.describe().T


,count,unique,top,freq,first,last
"Country,Other",4872,232,World,21,NaT,NaT
TotalCases,4872,1526,"18,491",21,NaT,NaT
NewCases,4872,666,,3597,NaT,NaT
TotalDeaths,4872,850,,126,NaT,NaT
NewDeaths,4872,167,,4255,NaT,NaT
TotalRecovered,4872,1412,N/A,285,NaT,NaT
NewRecovered,4872,655,,3397,NaT,NaT
ActiveCases,4872,1402,N/A,285,NaT,NaT
"Serious,Critical",4872,167,,2062,NaT,NaT
TotCases/1M pop,4872,1305,,42,NaT,NaT


Comment: 
- `worldometer.info/coronavirus/` provides the data of 229 countries and 2 other. The other is the Diamond Princess cruise ship and MS Zaandam cruise ship. Remember that we also have 1 row for the world so the total number of `Country,Other` is 232. The cruise ships are not countries so they are not included in any continent. So we have to remove them from the DataFrame.
- There are a kind of anomaly in `Continent`. There are 10 unique values althought we just have 7 continents

In [9]:
big_df.Continent.unique()

array(['All', 'Asia', 'North America', 'Europe', 'South America',
       'Australia/Oceania', 'Africa', 'Africa    ', 'Africa  ', ''],
      dtype=object)

There are more than 1 **Africa** and 1 empty `Continent`. We will have to handle it.

---
# Data preprocessing.

## Convert the columns to numeric.
If we have read the original dataset, we can relize that there were 2 kind of missing values: empty cells & N/A cells. Additionally, we can relize that empty cell means 0 (e.g there is a empty cell at `NewDeaths` means there is no new death), and N/A cells, they truely missing values.

In [10]:
header = big_df.columns.to_list()[1:-2]
header


['TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'NewRecovered',
 'ActiveCases',
 'Serious,Critical',
 'TotCases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/1M pop',
 'Population']

In [11]:
# clear comma in number
big_df[header] = big_df[header].apply(lambda x: x.str.replace(',', ''))
# convert empty cell is 0
big_df[header] = big_df[header].replace('', 0)
# convert string "N/A" to nan value
big_df[header] = big_df[header].replace("N/A", np.nan)
# convert object to numeric
big_df[header] = big_df[header].apply(pd.to_numeric)


In [12]:
# get DataFrame statistics
big_df.describe().T

,count,mean,std,min,25%,50%,75%,max
TotalCases,4872.0,5.869917e+06,4.543091e+07,5.0,24480.00,207869.5,1314562.50,6.823645e+08
NewCases,4872.0,7.769064e+02,6.347475e+03,0.0,0.00,0.0,2.00,1.346560e+05
TotalDeaths,4872.0,5.868406e+04,4.562075e+05,0.0,181.25,1968.5,14204.50,6.818745e+06
NewDeaths,4872.0,4.221264e+00,3.553143e+01,0.0,0.00,0.0,0.00,7.000000e+02
TotalRecovered,4587.0,5.899363e+06,4.494581e+07,0.0,17264.00,185101.0,1452118.50,6.552776e+08
NewRecovered,4587.0,1.222934e+03,1.005276e+04,0.0,0.00,0.0,1.00,2.093070e+05
ActiveCases,4587.0,1.788465e+05,1.576377e+06,-1548.0,43.00,1001.0,10152.00,2.044663e+07
"Serious,Critical",4722.0,2.566804e+02,2.738124e+03,0.0,0.00,2.0,19.00,4.067100e+04
TotCases/1M pop,4872.0,1.937057e+05,1.964549e+05,0.0,16823.25,123539.0,313120.25,7.039590e+05
Deaths/1M pop,4872.0,1.201995e+03,1.271577e+03,0.0,133.75,755.0,1949.00,6.520000e+03


In [13]:
big_df.describe(include=['O'])

,"Country,Other",Continent
count,4872,4872
unique,232,10
top,World,Africa
freq,21,1216


In [14]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4872 entries, 0 to 4871
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Country,Other     4872 non-null   object        
 1   TotalCases        4872 non-null   int64         
 2   NewCases          4872 non-null   int64         
 3   TotalDeaths       4872 non-null   int64         
 4   NewDeaths         4872 non-null   float64       
 5   TotalRecovered    4587 non-null   float64       
 6   NewRecovered      4587 non-null   float64       
 7   ActiveCases       4587 non-null   float64       
 8   Serious,Critical  4722 non-null   float64       
 9   TotCases/1M pop   4872 non-null   int64         
 10  Deaths/1M pop     4872 non-null   float64       
 11  TotalTests        4872 non-null   int64         
 12  Tests/1M pop      4872 non-null   int64         
 13  Population        4872 non-null   int64         
 14  Continent         4872 n

## Handle missing values

In [15]:
# calculate missing percentage
missing_percentage = big_df.isnull().mean() * 100
missing_percentage[missing_percentage.values > 0].sort_values(ascending=False)

TotalRecovered      5.849754
NewRecovered        5.849754
ActiveCases         5.849754
Serious,Critical    3.078818
dtype: float64

Comment: we will fill missing values with mean or median

Filling the missing values with the mean of the column.

In [16]:
%%capture
big_df = big_df.fillna(big_df.mean())

Now checking missing values again!

In [17]:
big_df.isnull().sum()

Country,Other       0
TotalCases          0
NewCases            0
TotalDeaths         0
NewDeaths           0
TotalRecovered      0
NewRecovered        0
ActiveCases         0
Serious,Critical    0
TotCases/1M pop     0
Deaths/1M pop       0
TotalTests          0
Tests/1M pop        0
Population          0
Continent           0
Date                0
dtype: int64

## Handle `Continent` features

In [18]:
big_df.Continent = big_df.Continent.str.strip()
big_df.Continent = big_df.Continent.replace('', 'All')

big_df.Continent.unique()

array(['All', 'Asia', 'North America', 'Europe', 'South America',
       'Australia/Oceania', 'Africa'], dtype=object)

In [19]:
big_df = big_df.drop(index=big_df[big_df['Continent'] == 'All'].index)
big_df.to_csv('covid19.csv', index=False)
big_df[big_df['Date'] > '2023-03-10'].to_csv('test.csv',index=False)
big_df[big_df['Date']<= '2023-03-10'].to_csv('train.csv',index=False)

In [20]:
data = pd.read_csv('train.csv')
features = ['TotalCases', 'TotalRecovered', 'TotalDeaths', 'NewDeaths', 'NewRecovered']
target = ['NewCases']
X = data[features]
Y = data[target]

train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.2, random_state=0)

In [21]:
test = pd.read_csv('test.csv')
test_X = test[['TotalCases', 'TotalRecovered', 'TotalDeaths', 'NewDeaths', 'NewRecovered']]
test_Y = test['NewCases']

In [22]:
regressor = DecisionTreeRegressor(random_state = 0) 
regressor.fit(train_X, train_Y)

lin = LinearRegression()
lin.fit(train_X,train_Y)

print(lin.score(val_X, val_Y)) # R-squared
print(regressor.score(val_X, val_Y)) # R-squared

0.7093474262065791
0.8819431574252362


In [23]:
res = pd.DataFrame({
    'Actual': test_Y,
    'Predicted Tree': regressor.predict(test_X).reshape(-1).astype(int),
    'Predicted Linear': lin.predict(test_X).reshape(-1).astype(int),
    'Country': test['Country,Other'],
    'Continent': test['Continent'],
    'Date': test['Date'],
    'R2 predicted Tree': regressor.score(test_X, test_Y),
    'R2 predicted Linear': lin.score(test_X, test_Y)
})

res.to_csv('result.csv', index=False)

In [24]:
print(lin.score(test_X, test_Y)) # R-squared
print(regressor.score(test_X, test_Y)) # R-squared

0.48780932361634277
0.5571534259922764


In [27]:
big_df.corr().to_csv('corr.csv')